In [ ]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

openjdk version "11.0.26" 2025-01-21
OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.11/dist-packages/pyspark"


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BDAV").getOrCreate()


In [ ]:
df = spark.read.csv("/content/games.csv", header=True, inferSchema=True)
df = df.fillna({'description':'NA'})
df = df.dropna()
df.show(100)

+-------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+
|     id|                name|      date|rating|reviews|plays|playing|backlogs|wishlists|         description|
+-------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+
|1000001|Cathode Ray Tube ...|1947-12-31|   3.6|     85|  149|      1|      42|       72|The cathode ray t...|
|1000002|    Bertie the Brain|1950-08-25|   3.0|     26|   46|      0|       9|       17|Currently conside...|
|1000003|                 Nim|1951-12-31|   1.9|      9|   26|      0|       2|        8|The Nimrod was a ...|
|1000004|            Draughts|1952-08-31|   2.8|      9|   30|      0|       4|        7|"A game of draugh...|
|1000005|                 OXO|1952-12-31|   3.1|     22|   80|      0|      11|       15|OXO was a compute...|
|1000006|                Pool|1954-06-26|   3.1|     12|   33|      0|       3|        4|"A game of pool (...|
|

In [ ]:
df = df.filter((df["rating"] >= 1.0) & (df["rating"] <= 5.0))
df.show(10)

+-------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+
|     id|                name|      date|rating|reviews|plays|playing|backlogs|wishlists|         description|
+-------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+
|1000001|Cathode Ray Tube ...|1947-12-31|   3.6|     85|  149|      1|      42|       72|The cathode ray t...|
|1000002|    Bertie the Brain|1950-08-25|   3.0|     26|   46|      0|       9|       17|Currently conside...|
|1000003|                 Nim|1951-12-31|   1.9|      9|   26|      0|       2|        8|The Nimrod was a ...|
|1000004|            Draughts|1952-08-31|   2.8|      9|   30|      0|       4|        7|"A game of draugh...|
|1000005|                 OXO|1952-12-31|   3.1|     22|   80|      0|      11|       15|OXO was a compute...|
|1000006|                Pool|1954-06-26|   3.1|     12|   33|      0|       3|        4|"A game of pool (...|
|

In [ ]:
filtered_df = df.filter(df["description"] == "NA")
filtered_df.show(10)

+-------+--------------------+----------+------+-------+-----+-------+--------+---------+-----------+
|     id|                name|      date|rating|reviews|plays|playing|backlogs|wishlists|description|
+-------+--------------------+----------+------+-------+-----+-------+--------+---------+-----------+
|1000217|          Pool Shark|1977-06-01|   1.8|      0|   16|      0|       4|        0|         NA|
|1000226|         M-79 Ambush|1977-07-01|   2.0|      1|   13|      0|       3|        0|         NA|
|1000236|              Safari|1977-09-01|   1.0|      1|   12|      0|       2|        0|         NA|
|1000261|                 M-4|1977-12-31|   2.4|      1|   12|      0|       4|        1|         NA|
|1000375|Math-A-Magic! / E...|1978-12-31|   1.6|      1|   12|      0|       2|        0|         NA|
|1000630|              Pulsar|1980-12-01|   2.9|      1|   14|      0|       3|        0|         NA|
|1000650|        Steeplechase|1980-12-31|   1.7|      3|   15|      0|       6|   

In [ ]:
grouped_df = df.groupBy("description").count()
grouped_df.show()

+--------------------+-----+
|         description|count|
+--------------------+-----+
|Demolition Herby ...|    1|
|In Super Bee the ...|    2|
|The Isle of Gelno...|    2|
|"In part two of t...|    1|
|Mervyn the sorcer...|    1|
|In Gain Ground, p...|    1|
|High-speed boat c...|    1|
|Devilman is a sid...|    1|
|Something was ter...|    1|
|An adult version ...|    1|
|You are T-800 in ...|    1|
|You have come to ...|    1|
|Super Airwolf is ...|    1|
|"Magic Pockets is...|    3|
|Ganbare Goemon Ga...|    1|
|SimLife was a gen...|    1|
|The sequel to Win...|    2|
|Coinciding with t...|    2|
|Dragon's Revenge ...|    2|
|Shadowrun is an a...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
transformed_df = df.filter(df["plays"] > 50).groupBy("rating").agg({"reviews": "avg"})
transformed_df.show()

+------+------------------+
|rating|      avg(reviews)|
+------+------------------+
|   1.0|              41.4|
|   2.6| 34.25430463576159|
|   3.1|  46.4198250728863|
|   4.2|266.47305389221555|
|   2.7|28.666666666666668|
|   1.7|18.031645569620252|
|   3.8|118.23697650663942|
|   4.4|354.81111111111113|
|   2.9|41.638632162661736|
|   4.5|           551.328|
|   2.4| 33.38814531548757|
|   2.5|29.851963746223564|
|   1.6| 39.93636363636364|
|   3.4| 68.13750899928006|
|   1.8| 25.02173913043478|
|   3.3| 52.32396345748419|
|   3.5|  70.2762962962963|
|   4.3|449.20647773279353|
|   4.8| 17.88888888888889|
|   4.1| 266.2888888888889|
+------+------------------+
only showing top 20 rows



In [ ]:
df.show()

+-------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+
|     id|                name|      date|rating|reviews|plays|playing|backlogs|wishlists|         description|
+-------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+
|1000001|Cathode Ray Tube ...|1947-12-31|   3.6|     85|  149|      1|      42|       72|The cathode ray t...|
|1000002|    Bertie the Brain|1950-08-25|   3.0|     26|   46|      0|       9|       17|Currently conside...|
|1000003|                 Nim|1951-12-31|   1.9|      9|   26|      0|       2|        8|The Nimrod was a ...|
|1000004|            Draughts|1952-08-31|   2.8|      9|   30|      0|       4|        7|"A game of draugh...|
|1000005|                 OXO|1952-12-31|   3.1|     22|   80|      0|      11|       15|OXO was a compute...|
|1000006|                Pool|1954-06-26|   3.1|     12|   33|      0|       3|        4|"A game of pool (...|
|

In [ ]:
developers_df = spark.read.csv("/content/developers.csv", header=True, inferSchema=True)
platforms_df = spark.read.csv("/content/platforms.csv", header=True, inferSchema=True)
genres_df = spark.read.csv("/content/genres.csv", header=True, inferSchema=True)
scores_df = spark.read.csv("/content/scores.csv", header=True, inferSchema=True)

In [ ]:
scores_df.show(10)
genres_df.show(10)
platforms_df.show(10)
developers_df.show(10)

+-------+-----+------+
|     id|score|amount|
+-------+-----+------+
|1000001|  0.5|     6|
|1000001|  1.0|     5|
|1000001|  1.5|     1|
|1000001|  2.0|     5|
|1000001|  2.5|    11|
|1000001|  3.0|    12|
|1000001|  3.5|     3|
|1000001|  4.0|     2|
|1000001|  4.5|     3|
|1000001|  5.0|    43|
+-------+-----+------+
only showing top 10 rows

+-------+-----------------+
|     id|            genre|
+-------+-----------------+
|1000001|  Point-and-Click|
|1000002|           Puzzle|
|1000002|         Tactical|
|1000003|          Pinball|
|1000003|         Strategy|
|1000004|Card & Board Game|
|1000005|           Puzzle|
|1000005|         Strategy|
|1000006|            Sport|
|1000007|           Arcade|
+-------+-----------------+
only showing top 10 rows

+-------+--------------------+
|     id|            platform|
+-------+--------------------+
|1000001|Analogue electronics|
|1000002|              Arcade|
|1000003|Ferranti Nimrod C...|
|1000004|     Legacy Computer|
|1000005|        

In [ ]:
from pyspark.sql.functions import sum, avg
from pyspark.sql.functions import collect_set, concat_ws
from pyspark.sql.functions import col

# Add a new column by multiplying 'score' and 'amount'
aggregated_scores = scores_df.withColumn("score_x_amount", col("score") * col("amount"))
aggregated_scores.show()

# Group by 'id' and calculate the sum of the new column
aggregated_scores = aggregated_scores.groupBy("id").agg(avg("score_x_amount").alias('Avg_Score'))
aggregated_scores = aggregated_scores.orderBy("id", ascending=True)

# Aggregate the DataFrame to calculate total amount and average score
genres_combined_df = genres_df.groupBy("id") \
    .agg(collect_set("genre").alias("genres")) \
    .withColumn("genres", concat_ws(", ", "genres"))

genres_combined_df.show()
aggregated_scores.show()

+-------+-----+------+--------------+
|     id|score|amount|score_x_amount|
+-------+-----+------+--------------+
|1000001|  0.5|     6|           3.0|
|1000001|  1.0|     5|           5.0|
|1000001|  1.5|     1|           1.5|
|1000001|  2.0|     5|          10.0|
|1000001|  2.5|    11|          27.5|
|1000001|  3.0|    12|          36.0|
|1000001|  3.5|     3|          10.5|
|1000001|  4.0|     2|           8.0|
|1000001|  4.5|     3|          13.5|
|1000001|  5.0|    43|         215.0|
|1000002|  0.5|     0|           0.0|
|1000002|  1.0|     3|           3.0|
|1000002|  1.5|     0|           0.0|
|1000002|  2.0|     4|           8.0|
|1000002|  2.5|     1|           2.5|
|1000002|  3.0|     1|           3.0|
|1000002|  3.5|     2|           7.0|
|1000002|  4.0|     0|           0.0|
|1000002|  4.5|     0|           0.0|
|1000002|  5.0|     4|          20.0|
+-------+-----+------+--------------+
only showing top 20 rows

+-------+--------------------+
|     id|              genres|


In [ ]:
platforms_combined_df = platforms_df.groupBy("id").agg(collect_set("platform").alias("platforms")).withColumn("platforms", concat_ws(", ", "platforms"))


In [ ]:
final_df = df.join(developers_df, on="id", how="inner")
final_df = aggregated_scores.join(final_df, on="id", how="inner")
final_df = genres_combined_df.join(final_df, on="id", how="inner")
final_df = platforms_combined_df.join(final_df, on="id", how="inner")


In [ ]:
final_df = final_df.dropDuplicates()
final_df.show(10)

+-------+--------------------+------------------+---------+----------------+----------+------+-------+-----+-------+--------+---------+--------------------+--------------------+
|     id|           platforms|            genres|Avg_Score|            name|      date|rating|reviews|plays|playing|backlogs|wishlists|         description|           developer|
+-------+--------------------+------------------+---------+----------------+----------+------+-------+-----+-------+--------+---------+--------------------+--------------------+
|1000002|              Arcade|  Tactical, Puzzle|     4.35|Bertie the Brain|1950-08-25|   3.0|     26|   46|      0|       9|       17|Currently conside...|         Josef Kates|
|1000005|   EDSAC, Windows PC|  Strategy, Puzzle|     11.5|             OXO|1952-12-31|   3.1|     22|   80|      0|      11|       15|OXO was a compute...|University of War...|
|1000005|   EDSAC, Windows PC|  Strategy, Puzzle|     11.5|             OXO|1952-12-31|   3.1|     22|   80|  

In [ ]:
final_df.write.csv("/content/final", header=True, mode = 'overwrite')

In [ ]:
final_df = df.join(developers_df, on="id", how="inner")
final_df = aggregated_scores.join(final_df, on="id", how="inner")
final_df = genres_df.join(final_df, on="id", how="inner")
final_df = platforms_df.join(final_df, on="id", how="inner")

In [ ]:
final_df.show(10)

+-------+--------------------+-------+---------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+--------------+
|     id|            platform|  genre|Avg_Score|                name|      date|rating|reviews|plays|playing|backlogs|wishlists|         description|     developer|
+-------+--------------------+-------+---------+--------------------+----------+------+-------+-----+-------+--------+---------+--------------------+--------------+
|1001043|Philips Videopac ...|Shooter|      0.8|                UFO!|1981-12-31|   2.7|      1|    6|      0|       0|        1|The game features...|      Magnavox|
|1001129|        Game & Watch| Arcade|     5.85|       Turtle Bridge|1982-02-01|   2.6|      6|   50|      0|       9|        2|Turtle Bridge is ...|      Nintendo|
|1002431|    Commodore VIC-20|    RPG|      0.3|Ultima: Escape fr...|1983-12-31|   1.0|      1|    9|      1|      11|        6|The odd one out o...|Sierra On-Line|
|1003031| 

In [ ]:
pivoted_df = final_df.groupBy("genre").pivot("platform").agg(avg("Avg_Score"))
pivoted_df.show(10)


+-------------------+---------------------------------------+---------------------------+-----------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------+------------------+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+--------------+------------+------------------+-----------------+------------------+--------------+-------------+------------------+-----------------+--------+------------------+--------+---------+---------------+------------------+------------------+-----+---------------------+---------------------------+------------------+------------------+-----------------+-------------------+-------------------------+---------

In [ ]:
pivoted_df_2 = final_df.groupBy("developer").pivot("genre").agg(avg("Avg_Score"))
pivoted_df_2.show(20)

+--------------------+------------------+------------------+-------+-----------------+-----------------+------------------+----+-----+-------+------------------+------------------+------------------+-----------+------------------+------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+-----------------+
|           developer|         Adventure|            Arcade|Brawler|Card & Board Game|         Fighting|             Indie|MOBA|Music|Pinball|          Platform|   Point-and-Click|            Puzzle|Quiz/Trivia|               RPG|Racing|Real Time Strategy|           Shooter|         Simulator|             Sport|          Strategy|         Tactical|Turn Based Strategy|     Visual Novel|
+--------------------+------------------+------------------+-------+-----------------+-----------------+------------------+----+-----+-------+------------------+------------------+------------------+-----------+-----------

In [ ]:
pivoted_df_2 = pivoted_df_2.fillna(0)
pivoted_df_2.write.csv("/content/pivot", header=True, mode = 'overwrite')

In [ ]:
import time

start_time = time.time()
final_df.groupBy("developer").pivot("genre").agg(avg("Avg_Score")).show()  # Trigger action for timing
end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

+--------------------+------------------+------------------+-------+-----------------+-----------------+------------------+----+-----+-------+------------------+------------------+------------------+-----------+------------------+------+------------------+------------------+------------------+------------------+------------------+-----------------+-------------------+-----------------+
|           developer|         Adventure|            Arcade|Brawler|Card & Board Game|         Fighting|             Indie|MOBA|Music|Pinball|          Platform|   Point-and-Click|            Puzzle|Quiz/Trivia|               RPG|Racing|Real Time Strategy|           Shooter|         Simulator|             Sport|          Strategy|         Tactical|Turn Based Strategy|     Visual Novel|
+--------------------+------------------+------------------+-------+-----------------+-----------------+------------------+----+-----+-------+------------------+------------------+------------------+-----------+-----------

In [ ]:
from pyspark.sql.functions import col, when, lit

unpivoted_df = pivoted_df_2.selectExpr("developer", "stack(24, 'Adventure', Adventure, 'Arcade', Arcade, 'Brawler', Brawler, 'Card & Board Game', `Card & Board Game`, 'Fighting', Fighting, 'Indie', Indie, 'MOBA', MOBA, 'Music', Music, 'Pinball', Pinball, 'Platform', Platform, 'Point-and-Click', `Point-and-Click`, 'Puzzle', Puzzle, 'Quiz/Trivia', `Quiz/Trivia`, 'RPG', RPG, 'Racing', Racing, 'Real Time Strategy', `Real Time Strategy`, 'Shooter', Shooter, 'Simulator', Simulator, 'Sport', Sport, 'Strategy', Strategy, 'Tactical', Tactical, 'Turn Based Strategy', `Turn Based Strategy`, 'Visual Novel', `Visual Novel`) as (genre, Avg_Score)").where('Avg_Score is not null')

unpivoted_df.show()


+--------------+------------------+------------------+
|     developer|             genre|         Avg_Score|
+--------------+------------------+------------------+
|11 bit studios|         Adventure|273.84829545454573|
|11 bit studios|            Arcade|               4.3|
|11 bit studios|           Brawler|            972.75|
|11 bit studios| Card & Board Game|               0.0|
|11 bit studios|          Fighting|               0.0|
|11 bit studios|             Indie|265.20409090909095|
|11 bit studios|              MOBA|               0.0|
|11 bit studios|             Music|               0.0|
|11 bit studios|           Pinball|               0.0|
|11 bit studios|          Platform|               0.0|
|11 bit studios|   Point-and-Click| 86.44999999999999|
|11 bit studios|            Puzzle|101.97999999999999|
|11 bit studios|       Quiz/Trivia|               0.0|
|11 bit studios|               RPG|438.21714285714324|
|11 bit studios|            Racing|               0.0|
|11 bit st